In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
from IPython.display import display, Markdown
from agents import OpenAIChatCompletionsModel, AsyncOpenAI
import os

load_dotenv(override=True)

True

In [5]:
from accounts import Account

In [6]:
account = Account.get("Anand")
account

Account(name='anand', balance=10000.0, strategy='', holdings={}, transactions=[], portfolio_value_time_series=[])

In [7]:
account.buy_shares("AMZN", 3, "Because this bookstore website looks promising")

'Completed. Latest details:\n{"name": "anand", "balance": 9801.604, "strategy": "", "holdings": {"AMZN": 3}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 66.132, "timestamp": "2025-05-30 10:19:27", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-05-30 10:19:27", 9903.604]], "total_portfolio_value": 9903.604, "total_profit_loss": -96.39600000000064}'

In [8]:
account.report()

'{"name": "anand", "balance": 9801.604, "strategy": "", "holdings": {"AMZN": 3}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 66.132, "timestamp": "2025-05-30 10:19:27", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-05-30 10:19:27", 9903.604], ["2025-05-30 10:19:29", 10071.604]], "total_portfolio_value": 10071.604, "total_profit_loss": 71.60399999999936}'

In [9]:
account.list_transactions()

[{'symbol': 'AMZN',
  'quantity': 3,
  'price': 66.132,
  'timestamp': '2025-05-30 10:19:27',
  'rationale': 'Because this bookstore website looks promising'}]

### Now we write an MCP server and use it directly!

In [12]:
# Now let's use our accounts server as an MCP server

params = {"command": "uv", "args": ["run", "accounts_server.py"]}
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
    mcp_tools = await server.list_tools()

In [13]:
mcp_tools

[Tool(name='get_balance', description='Get the cash balance of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_balanceArguments', 'type': 'object'}, annotations=None),
 Tool(name='get_holdings', description='Get the holdings of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_holdingsArguments', 'type': 'object'}, annotations=None),
 Tool(name='buy_shares', description="Buy shares of a stock.\n\n    Args:\n        name: The name of the account holder\n        symbol: The symbol of the stock\n        quantity: The quantity of shares to buy\n        rationale: The rationale for the purchase and fit with the account's strategy\n    ", inputSchema={'properties': {'name': {'title': 'Name', '

In [14]:
instructions = "You are able to manage an account for a client, and answer questions about the account."
request = "My name is Anand and my account is under the name Anand. What's my balance and my holdings?"

gemini_model = OpenAIChatCompletionsModel(
    model="gemini-2.0-flash",
    openai_client=AsyncOpenAI(
        base_url=os.getenv("GEMINI_URL"),  # Ollama's OpenAI-compatible endpoint
        api_key=os.getenv("GEMINI_API_KEY"),  # Any string works, it's not authenticated
    ),
)

In [15]:
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="account_manager", instructions=instructions, model=gemini_model, mcp_servers=[mcp_server])
    with trace("account_manager"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

OK. Your cash balance is 9801.604 and you have 3 shares of AMZN.

### Now let's build our own MCP Client

In [16]:
from accounts_client import get_accounts_tools_openai, read_accounts_resource, list_accounts_tools

mcp_tools = await list_accounts_tools()
print(mcp_tools)
openai_tools = await get_accounts_tools_openai()
print(openai_tools)

[Tool(name='get_balance', description='Get the cash balance of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_balanceArguments', 'type': 'object'}, annotations=None), Tool(name='get_holdings', description='Get the holdings of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_holdingsArguments', 'type': 'object'}, annotations=None), Tool(name='buy_shares', description="Buy shares of a stock.\n\n    Args:\n        name: The name of the account holder\n        symbol: The symbol of the stock\n        quantity: The quantity of shares to buy\n        rationale: The rationale for the purchase and fit with the account's strategy\n    ", inputSchema={'properties': {'name': {'title': 'Name', 'ty

In [18]:
request = "My name is Anand and my account is under the name Anand. What's my balance?"

with trace("account_mcp_client"):
    agent = Agent(name="account_manager", instructions=instructions, model=gemini_model, tools=openai_tools)
    result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

OK. Your balance is $9801.604.

In [19]:
context = await read_accounts_resource("Anand")
print(context)

{"name": "anand", "balance": 9801.604, "strategy": "", "holdings": {"AMZN": 3}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 66.132, "timestamp": "2025-05-30 10:19:27", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-05-30 10:19:27", 9903.604], ["2025-05-30 10:19:29", 10071.604], ["2025-05-30 10:23:50", 9933.604]], "total_portfolio_value": 9933.604, "total_profit_loss": -66.39600000000064}


In [20]:
from accounts import Account
Account.get("Anand").report()

'{"name": "anand", "balance": 9801.604, "strategy": "", "holdings": {"AMZN": 3}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 66.132, "timestamp": "2025-05-30 10:19:27", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-05-30 10:19:27", 9903.604], ["2025-05-30 10:19:29", 10071.604], ["2025-05-30 10:23:50", 9933.604], ["2025-05-30 10:23:50", 9963.604]], "total_portfolio_value": 9963.604, "total_profit_loss": -36.39600000000064}'